In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import Dataset, DataLoader
import spacy
import os
import random
from collections import Counter
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [6]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
# !pip install torch


# ============================================================================
# 1. CHUẨN BỊ DỮ LIỆU
# ============================================================================

In [ ]:
class Vocabulary:
    """Xây dựng từ điển cho một ngôn ngữ"""
    def __init__(self, max_vocab_size=10000):
        self.max_vocab_size = max_vocab_size
        self.word2idx = {}
        self.idx2word = {}
        self.word_freq = Counter()

        # Token đặc biệt
        self.PAD_TOKEN = '<pad>'
        self.UNK_TOKEN = '<unk>'
        self.SOS_TOKEN = '<sos>'
        self.EOS_TOKEN = '<eos>'

        self.pad_idx = 0
        self.unk_idx = 1
        self.sos_idx = 2
        self.eos_idx = 3

    def build_vocab(self, sentences):
        """Xây dựng từ điển từ danh sách câu"""
        # Đếm tần suất từ
        for sentence in sentences:
            self.word_freq.update(sentence)

        # Lấy top từ phổ biến
        most_common = self.word_freq.most_common(self.max_vocab_size - 4)

        # Thêm token đặc biệt
        self.word2idx = {
            self.PAD_TOKEN: self.pad_idx,
            self.UNK_TOKEN: self.unk_idx,
            self.SOS_TOKEN: self.sos_idx,
            self.EOS_TOKEN: self.eos_idx
        }

        # Thêm các từ phổ biến
        for idx, (word, freq) in enumerate(most_common, start=4):
            self.word2idx[word] = idx

        # Tạo ánh xạ ngược
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

    def encode(self, sentence):
        """Chuyển câu thành list index"""
        return [self.word2idx.get(word, self.unk_idx) for word in sentence]

    def decode(self, indices):
        """Chuyển list index thành câu"""
        return [self.idx2word.get(idx, self.UNK_TOKEN) for idx in indices]

    def __len__(self):
        return len(self.word2idx)


class TranslationDataset(Dataset):
    """Dataset cho dịch máy"""
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab):
        self.src_sentences = src_sentences
        self.tgt_sentences = tgt_sentences
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, idx):
        src = self.src_sentences[idx]
        tgt = self.tgt_sentences[idx]

        # Encode thành indices
        src_indices = self.src_vocab.encode(src) + [self.src_vocab.eos_idx]
        tgt_indices = [self.tgt_vocab.sos_idx] + self.tgt_vocab.encode(tgt) + [self.tgt_vocab.eos_idx]

        return torch.LongTensor(src_indices), torch.LongTensor(tgt_indices)


def collate_fn(batch):
    """Collate function để xử lý padding và sorting"""
    src_batch, tgt_batch = zip(*batch)

    # Sắp xếp theo độ dài giảm dần (bắt buộc cho pack_padded_sequence)
    src_lengths = torch.LongTensor([len(s) for s in src_batch])
    sorted_indices = src_lengths.argsort(descending=True)

    src_batch = [src_batch[i] for i in sorted_indices]
    tgt_batch = [tgt_batch[i] for i in sorted_indices]
    src_lengths = src_lengths[sorted_indices]

    # Padding
    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=0)

    return src_padded, src_lengths, tgt_padded

# ============================================================================
# 2. MÔ HÌNH ENCODER-DECODER
# ============================================================================

In [ ]:


class Encoder(nn.Module):
    """LSTM Encoder"""
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lengths):
        """
        Args:
            src: (batch_size, src_len)
            src_lengths: (batch_size,)
        Returns:
            hidden: (num_layers, batch_size, hidden_size)
            cell: (num_layers, batch_size, hidden_size)
        """
        embedded = self.dropout(self.embedding(src))  # (batch, src_len, emb_dim)

        # Pack sequence
        packed = pack_padded_sequence(embedded, src_lengths.cpu(), batch_first=True, enforce_sorted=True)

        # LSTM
        packed_output, (hidden, cell) = self.lstm(packed)

        # hidden: (num_layers, batch, hidden_size)
        # cell: (num_layers, batch, hidden_size)
        return hidden, cell


class Decoder(nn.Module):
    """LSTM Decoder"""
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers,
                           batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc_out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_token, hidden, cell):
        """
        Args:
            input_token: (batch_size, 1)
            hidden: (num_layers, batch_size, hidden_size)
            cell: (num_layers, batch_size, hidden_size)
        Returns:
            prediction: (batch_size, vocab_size)
            hidden: (num_layers, batch_size, hidden_size)
            cell: (num_layers, batch_size, hidden_size)
        """
        embedded = self.dropout(self.embedding(input_token))  # (batch, 1, emb_dim)

        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # output: (batch, 1, hidden_size)

        prediction = self.fc_out(output.squeeze(1))  # (batch, vocab_size)

        return prediction, hidden, cell


# Thay thế class Seq2Seq.forward bằng đoạn này
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_lengths, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        tgt_len = tgt.shape[1]
        tgt_vocab_size = self.decoder.vocab_size

        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)

        # Encoder
        hidden, cell = self.encoder(src, src_lengths)

        # Nếu dùng full teacher forcing -> vectorized decoding (1 pass LSTM)
        if teacher_forcing_ratio >= 1.0 - 1e-9:
            # Input to decoder LSTM are the target tokens excluding the last token
            # (we use targets as inputs when teacher forcing)
            # tgt_in: shape (batch, tgt_len-1)
            tgt_in = tgt[:, :-1]  # exclude final <eos> if present
            embedded = self.decoder.embedding(tgt_in)  # (batch, seq_len, emb_dim)
            embedded = self.decoder.dropout(embedded)

            # Run decoder LSTM once for the whole sequence
            decoder_outputs, (hidden, cell) = self.decoder.lstm(embedded, (hidden, cell))
            # decoder_outputs: (batch, seq_len, hidden_size)
            # Map to vocab
            pred = self.decoder.fc_out(decoder_outputs)  # (batch, seq_len, vocab_size)

            # Place predictions into outputs aligned so that outputs[:, t, :] predicts token at tgt[:, t]
            outputs[:, 1:tgt_len, :] = pred  # note: pred corresponds to steps 1..tgt_len-1
            # (outputs[:,0,:] stays zeros or you can set it to prediction for <sos> if you want)
            return outputs

        # Fallback: mixed or no teacher forcing -> step-by-step (cũ)
        decoder_input = tgt[:, 0].unsqueeze(1)  # <sos>
        for t in range(1, tgt_len):
            prediction, hidden, cell = self.decoder(decoder_input, hidden, cell)
            outputs[:, t, :] = prediction
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = prediction.argmax(1).unsqueeze(1)
            decoder_input = tgt[:, t].unsqueeze(1) if teacher_force else top1

        return outputs


# ============================================================================
# 3. HUẤN LUYỆN
# ============================================================================


In [7]:
def train_epoch(model, dataloader, optimizer, criterion, clip, device):
    """Huấn luyện 1 epoch"""
    model.train()
    epoch_loss = 0

    for src, src_lengths, tgt in dataloader:
        src = src.to(device)
        tgt = tgt.to(device)

        optimizer.zero_grad()

        # Forward
        output = model(src, src_lengths, tgt, teacher_forcing_ratio=0.5)

        # output: (batch, tgt_len, vocab_size)
        # tgt: (batch, tgt_len)

        # Bỏ <sos> token và flatten
        output = output[:, 1:, :].reshape(-1, output.shape[-1])
        tgt = tgt[:, 1:].reshape(-1)

        # Tính loss
        loss = criterion(output, tgt)

        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)


def evaluate(model, dataloader, criterion, device):
    """Đánh giá trên tập validation"""
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for src, src_lengths, tgt in dataloader:
            src = src.to(device)
            tgt = tgt.to(device)

            # Forward (không teacher forcing)
            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0)

            output = output[:, 1:, :].reshape(-1, output.shape[-1])
            tgt = tgt[:, 1:].reshape(-1)

            loss = criterion(output, tgt)
            epoch_loss += loss.item()

    return epoch_loss / len(dataloader)

# ============================================================================
# 4. DỰ ĐOÁN (INFERENCE)
# ============================================================================


In [8]:
def translate_sentence(model, sentence, src_vocab, tgt_vocab, src_tokenizer, device, max_len=50):
    """
    Dịch một câu từ tiếng Anh sang tiếng Pháp

    Args:
        model: mô hình đã train
        sentence: câu tiếng Anh (string)
        src_vocab, tgt_vocab: từ điển
        src_tokenizer: tokenizer cho tiếng Anh
        device: cuda/cpu
        max_len: độ dài tối đa câu dịch

    Returns:
        translated_sentence: câu tiếng Pháp (string)
    """
    model.eval()

    # Tokenize
    tokens = src_tokenizer(sentence.lower())
    tokens = [token.text for token in tokens]

    # Encode
    indices = src_vocab.encode(tokens) + [src_vocab.eos_idx]
    src_tensor = torch.LongTensor(indices).unsqueeze(0).to(device)  # (1, src_len)
    src_lengths = torch.LongTensor([len(indices)])

    with torch.no_grad():
        # Encoder
        hidden, cell = model.encoder(src_tensor, src_lengths)

        # Decoder
        decoder_input = torch.LongTensor([tgt_vocab.sos_idx]).unsqueeze(0).to(device)
        translated_indices = []

        for _ in range(max_len):
            prediction, hidden, cell = model.decoder(decoder_input, hidden, cell)
            predicted_token = prediction.argmax(1).item()

            if predicted_token == tgt_vocab.eos_idx:
                break

            translated_indices.append(predicted_token)
            decoder_input = torch.LongTensor([predicted_token]).unsqueeze(0).to(device)

    # Decode
    translated_tokens = tgt_vocab.decode(translated_indices)
    translated_sentence = ' '.join(translated_tokens)

    return translated_sentence


def calculate_bleu(model, test_data, src_vocab, tgt_vocab, src_tokenizer, device):
    """Tính BLEU score trên tập test"""
    bleu_scores = []
    smoothing = SmoothingFunction().method1

    for src_sent, tgt_sent in test_data:
        src_text = ' '.join(src_sent)
        translated = translate_sentence(model, src_text, src_vocab, tgt_vocab, src_tokenizer, device)

        reference = [tgt_sent]
        candidate = translated.split()

        score = sentence_bleu(reference, candidate, smoothing_function=smoothing)
        bleu_scores.append(score)

    return np.mean(bleu_scores)


In [ ]:
# ---------------------
# 📁 Đường dẫn file dữ liệu
# ---------------------
en_path = r"D:\HK7NAM4\LTSM\English_French_Machine_Translation_LSTM\dataset\fr-en\europarl-v7.fr-en.en"
fr_path = r"D:\HK7NAM4\LTSM\English_French_Machine_Translation_LSTM\dataset\fr-en\europarl-v7.fr-en.fr"

# ---------------------
# 📄 Hàm đọc file
# ---------------------
def load_lines(path):
    lines = []
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            line = line.strip()
            if line:
                lines.append(line)
    return lines

# ---------------------
# 🔹 Đọc dữ liệu gốc
# ---------------------
print("🔹 Đang đọc dữ liệu gốc...")
en_lines = load_lines(en_path)
fr_lines = load_lines(fr_path)
print(f"Tổng số câu song ngữ: {len(en_lines):,}")

🔹 Đang đọc dữ liệu gốc...
Tổng số câu song ngữ: 2,005,688


1️⃣ Dòng trống hoặc whitespace

line.strip() loại bỏ khoảng trắng, và if line: bỏ các dòng rỗng.

Nếu file dataset có nhiều dòng trống, chúng sẽ bị loại bỏ.

2️⃣ Dòng không khớp giữa 2 file

File EN và file FR phải cùng số dòng, mỗi dòng tương ứng là cặp câu song ngữ.

Nếu một file có ít dòng hơn, bạn phải cắt bằng nhau bằng min(len(en_lines), len(fr_lines)).

Những dòng thừa ở file dài hơn sẽ bị loại bỏ.

3️⃣ Lỗi encoding / ký tự đặc biệt

Bạn dùng errors='ignore', nghĩa là những dòng có ký tự không đọc được sẽ bị bỏ.

Europarl có nhiều ký tự đặc biệt hoặc mã lỗi, nên nhiều dòng bị mất.

4️⃣ Chỉ giữ những dòng không rỗng và hợp lệ

Khi train_test_split chạy, chỉ những cặp dòng tồn tại ở cả 2 file mới được dùng → số lượng giảm mạnh.

In [17]:
# Hyperparameters
EMBEDDING_DIM = 256
HIDDEN_SIZE = 512
NUM_LAYERS = 2
DROPOUT = 0.5
LEARNING_RATE = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 10
CLIP = 1
PATIENCE = 3  # cho early stopping

In [ ]:
# Chọn số câu tối đa bằng độ dài nhỏ hơn
min_len = min(len(en_lines), len(fr_lines))
en_lines = en_lines[:min_len]
fr_lines = fr_lines[:min_len]

print(f"Số câu sau khi đồng bộ: {len(en_lines):,}")


NameError: name 'en_lines' is not defined

In [ ]:
# ---------------------
# ✂️ Chia train / val / test
# ---------------------
from sklearn.model_selection import train_test_split

train_en, temp_en, train_fr, temp_fr = train_test_split(
    en_lines, fr_lines, test_size=0.1, random_state=42
)
val_en, test_en, val_fr, test_fr = train_test_split(
    temp_en, temp_fr, test_size=0.5, random_state=42
)

print(f"Train set: {len(train_en):,} câu")
print(f"Validation set: {len(val_en):,} câu")
print(f"Test set: {len(test_en):,} câu")

Train set: 1,804,311 câu
Validation set: 100,240 câu
Test set: 100,240 câu


In [ ]:
# ---------------------
# 🔠 Tokenizer (SpaCy)
# ---------------------
import spacy

try:
    en_tokenizer = spacy.load("en_core_web_sm", disable=["parser", "ner", "tagger"])
except Exception:
    print("⚠️ en_core_web_sm not found → dùng spacy.blank('en')")
    en_tokenizer = spacy.blank("en")

try:
    fr_tokenizer = spacy.load("fr_core_news_sm", disable=["parser", "ner", "tagger"])
except Exception:
    print("⚠️ fr_core_news_sm not found → dùng spacy.blank('fr')")
    fr_tokenizer = spacy.blank("fr")

print("✅ Tokenizers loaded successfully!")


⚠️ en_core_web_sm not found → dùng spacy.blank('en')
⚠️ fr_core_news_sm not found → dùng spacy.blank('fr')
✅ Tokenizers loaded successfully!


In [ ]:
from tqdm import tqdm

def tokenize_lines_fast(lines, tokenizer, n_process=4, batch_size=1000):
    """
    Tokenize nhanh bằng spaCy.pipe (đa luồng, ít tốn RAM)
    """
    tokenized = []
    for doc in tqdm(
        tokenizer.pipe(lines, n_process=n_process, batch_size=batch_size),
        total=len(lines),
        desc="🔹 Tokenizing"
    ):
        toks = [t.text.lower() for t in doc if t.text.strip()]
        tokenized.append(toks)
    return tokenized

print("✅ Defined fast tokenization function.")


✅ Defined fast tokenization function.


In [ ]:
print("🔹 Tokenizing train set...")
train_en_tok = tokenize_lines_fast(train_en, en_tokenizer)
train_fr_tok = tokenize_lines_fast(train_fr, fr_tokenizer)

print("🔹 Tokenizing validation set...")
val_en_tok = tokenize_lines_fast(val_en, en_tokenizer)
val_fr_tok = tokenize_lines_fast(val_fr, fr_tokenizer)

print("Tokenizing test set...")
test_en_tok = tokenize_lines_fast(test_en, en_tokenizer)
test_fr_tok = tokenize_lines_fast(test_fr, fr_tokenizer)

print("Done tokenizing all datasets!")


🔹 Tokenizing train set...


🔹 Tokenizing: 100%|██████████| 1804311/1804311 [23:45<00:00, 1266.00it/s]


🔹 Tokenizing validation set...


🔹 Tokenizing: 100%|██████████| 100240/100240 [01:47<00:00, 929.23it/s] 


Tokenizing test set...


🔹 Tokenizing: 100%|██████████| 100240/100240 [01:43<00:00, 972.57it/s] 

Done tokenizing all datasets!


In [ ]:
import pickle

with open("tokenized_data.pkl", "wb") as f:
    pickle.dump({
        "train_en_tok": train_en_tok,
        "train_fr_tok": train_fr_tok,
        "val_en_tok": val_en_tok,
        "val_fr_tok": val_fr_tok,
        "test_en_tok": test_en_tok,
        "test_fr_tok": test_fr_tok
    }, f)

print("✅ Đã lưu tokenized dataset thành tokenized_data.pkl")

✅ Đã lưu tokenized dataset thành tokenized_data.pkl


In [10]:
import pickle

with open("D:/HK7NAM4/LTSM/tokenized_data.pkl", "rb") as f:
    data = pickle.load(f)

train_en_tok = data["train_en_tok"]
train_fr_tok = data["train_fr_tok"]
val_en_tok = data["val_en_tok"]
val_fr_tok = data["val_fr_tok"]
test_en_tok = data["test_en_tok"]
test_fr_tok = data["test_fr_tok"]

print("✅ Loaded tokenized dataset from Drive!")


✅ Loaded tokenized dataset from Drive!


In [18]:
# --------------------------
# 📉 Subset để train nhanh hơn
# --------------------------
subset_size = 50_000  # hoặc 300_000 nếu GPU bạn ổn

train_en_tok = train_en_tok[:subset_size]
train_fr_tok = train_fr_tok[:subset_size]

# (val và test có thể giữ nguyên, vì chúng nhỏ)
print(f"✅ Đã rút gọn tập train xuống {len(train_en_tok):,} câu.")


✅ Đã rút gọn tập train xuống 50,000 câu.


In [19]:
print("Building vocabularies...")

src_vocab = Vocabulary(max_vocab_size=10000)
src_vocab.build_vocab(train_en_tok)

tgt_vocab = Vocabulary(max_vocab_size=10000)
tgt_vocab.build_vocab(train_fr_tok)

print(f"English vocab size: {len(src_vocab)}")
print(f"French vocab size: {len(tgt_vocab)}")


Building vocabularies...
English vocab size: 10000
French vocab size: 10000


In [20]:
from torch.utils.data import DataLoader
from tqdm import tqdm

print("🔹 Creating datasets...")

train_dataset = TranslationDataset(train_en_tok, train_fr_tok, src_vocab, tgt_vocab)
val_dataset = TranslationDataset(val_en_tok, val_fr_tok, src_vocab, tgt_vocab)
test_data = list(zip(test_en_tok, test_fr_tok))  # cho BLEU

# hiển thị quá trình khởi tạo dataloader
print("🔹 Building dataloaders...")
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"✅ Train examples: {len(train_dataset):,}")
print(f"✅ Val examples: {len(val_dataset):,}")
print(f"✅ Test examples: {len(test_data):,}")

print("🔹 Example tokenized (src → tgt):")
print(train_en_tok[0][:15], "→", train_fr_tok[0][:15])


🔹 Creating datasets...
🔹 Building dataloaders...
✅ Train examples: 50,000
✅ Val examples: 100,240
✅ Test examples: 100,240
🔹 Example tokenized (src → tgt):
['i', 'read', 'them', 'both', 'with', 'great', 'interest', '.'] → ['je', 'suis', 'fondamentalement', 'réceptif', 'à', 'toutes', 'les', 'tendances', 'libéralisatrices', 'qui', 'rendent', 'possibles', 'de', 'nouvelles', 'formes']


In [21]:
encoder = Encoder(len(src_vocab), EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
decoder = Decoder(len(tgt_vocab), EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
model = Seq2Seq(encoder, decoder, device).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab.pad_idx)

print("✅ Model initialized and ready for training.")


✅ Model initialized and ready for training.


In [22]:
from tqdm import tqdm

def train_epoch(model, dataloader, optimizer, criterion, clip, device):
    model.train()
    epoch_loss = 0

    # leave=True để thanh tiến trình vẫn hiện sau khi xong
    loop = tqdm(dataloader, desc="🧠 Training", leave=True)
    for src, src_lengths, tgt in loop:  # Unpack src_lengths
        src, tgt = src.to(device), tgt.to(device)
        optimizer.zero_grad()

        # Forward (có teacher forcing)
        output = model(src, src_lengths, tgt, teacher_forcing_ratio=0.5)

        # output: (batch, tgt_len, vocab_size)
        # tgt: (batch, tgt_len)
        output = output[:, 1:, :].reshape(-1, output.shape[-1])
        tgt = tgt[:, 1:].reshape(-1)

        # Loss
        loss = criterion(output, tgt)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        epoch_loss += loss.item()
        loop.set_postfix(loss=f"{loss.item():.3f}")

    return epoch_loss / len(dataloader)


def evaluate(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        loop = tqdm(dataloader, desc="🔍 Evaluating", leave=True)
        for src, src_lengths, tgt in loop:
            src, tgt = src.to(device), tgt.to(device)
            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0)
            output = output[:, 1:, :].reshape(-1, output.shape[-1])
            tgt = tgt[:, 1:].reshape(-1)
            loss = criterion(output, tgt)
            epoch_loss += loss.item()
            loop.set_postfix(loss=f"{loss.item():.3f}")
    return epoch_loss / len(dataloader)


# === MAIN TRAIN LOOP ===
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(NUM_EPOCHS):
    print(f"\n🌍 Epoch {epoch+1}/{NUM_EPOCHS}")
    train_loss = train_epoch(model, train_loader, optimizer, criterion, CLIP, device)
    val_loss = evaluate(model, val_loader, criterion, device)

    # ✅ Kết quả gọn đẹp giống code cũ
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f}")

    # Lưu best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
        print("💾 → Best model saved!")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("⛔ Early stopping!")
            break



🌍 Epoch 1/10


🧠 Training:  53%|█████▎    | 417/782 [1:24:25<1:13:53, 12.15s/it, loss=5.681]


KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))
bleu_score = calculate_bleu(model, test_data, src_vocab, tgt_vocab, en_tokenizer, device)
print(f"✅ BLEU Score on test set: {bleu_score:.4f}")


# ============================================================================
# 5. MAIN - SỬ DỤNG
# ============================================================================

In [ ]:
# # Removed the main() function definition and moved the code outside
# # This makes the variables (model, vocabularies, tokenizer, device, etc.) available globally.

# # Cấu hình
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Hyperparameters
# EMBEDDING_DIM = 256
# HIDDEN_SIZE = 512
# NUM_LAYERS = 2
# DROPOUT = 0.5
# LEARNING_RATE = 0.001
# BATCH_SIZE = 64
# NUM_EPOCHS = 30
# CLIP = 1

# # -- Đường dẫn tới thư mục dữ liệu (tương đối với notebook) --
# data_dir = '/content/drive/MyDrive/Dataset'

# def load_lines(path):
#     """Đọc file và trả về danh sách dòng (không tokenized)"""
#     lines = []
#     with open(path, 'r', encoding='utf-8', errors='ignore') as f:
#         for line in f:
#             line = line.strip()
#             if line:
#                 lines.append(line)
#     return lines

# def tokenize_lines(lines, tokenizer):
#     """Tokenize danh sách câu trả về list of token lists"""
#     tokenized = []
#     for line in lines:
#         toks = [t.text for t in tokenizer(line.lower()) if t.text.strip()]
#         tokenized.append(toks)
#     return tokenized

# # Tập tin theo cấu trúc workspace: dataset/<split>/(file)
# train_en_path = os.path.join(data_dir, 'train.en')
# train_fr_path = os.path.join(data_dir, 'train.fr')
# val_en_path = os.path.join(data_dir, 'val.en')
# val_fr_path = os.path.join(data_dir, 'val.fr')
# test_en_path = os.path.join(data_dir, 'test_2018_flickr.en')
# test_fr_path = os.path.join(data_dir, 'test_2018_flickr.fr')


# # Load raw lines
# train_en_lines = load_lines(train_en_path)
# train_fr_lines = load_lines(train_fr_path)
# val_en_lines = load_lines(val_en_path)
# val_fr_lines = load_lines(val_fr_path)
# test_en_lines = load_lines(test_en_path)
# test_fr_lines = load_lines(test_fr_path)

# # Tokenizers: thử load spacy models, fallback sang spacy.blank nếu model chưa cài
# try:
#     en_tokenizer = spacy.load('en_core_web_sm')
# except Exception:
#     # Nếu model en_core_web_sm chưa cài, dùng blank tokenizer (đơn giản)
#     print("Warning: en_core_web_sm not found. Using spacy.blank('en'). Please install with !python -m spacy download en_core_web_sm")
#     en_tokenizer = spacy.blank('en')

# try:
#     fr_tokenizer = spacy.load('fr_core_news_sm')
# except Exception:
#     print("Warning: fr_core_news_sm not found. Using spacy.blank('fr'). Please install with !python -m spacy download fr_core_news_sm")
#     fr_tokenizer = spacy.blank('fr')


# # Tokenize tất cả
# print('Tokenizing...')
# train_en_tok = tokenize_lines(train_en_lines, en_tokenizer)
# train_fr_tok = tokenize_lines(train_fr_lines, fr_tokenizer)
# val_en_tok = tokenize_lines(val_en_lines, en_tokenizer)
# val_fr_tok = tokenize_lines(val_fr_lines, fr_tokenizer)
# test_en_tok = tokenize_lines(test_en_lines, en_tokenizer)
# test_fr_tok = tokenize_lines(test_fr_lines, fr_tokenizer)

# # Build vocabularies (từ train set)
# print('Building vocabularies...')
# src_vocab = Vocabulary(max_vocab_size=10000)
# src_vocab.build_vocab(train_en_tok)
# tgt_vocab = Vocabulary(max_vocab_size=10000)
# tgt_vocab.build_vocab(train_fr_tok)

# # Tạo Dataset và DataLoader
# train_dataset = TranslationDataset(train_en_tok, train_fr_tok, src_vocab, tgt_vocab)
# val_dataset = TranslationDataset(val_en_tok, val_fr_tok, src_vocab, tgt_vocab)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# # test_data dùng cho calculate_bleu (dạng list of (src_tokens, tgt_tokens))
# test_data = list(zip(test_en_tok, test_fr_tok))

# # Sanity prints
# print(f'Train examples: {len(train_dataset)}')
# print(f'Val examples: {len(val_dataset)}')
# print(f'Test examples: {len(test_data)}')
# print('Example tokenized (src -> tgt):')
# print(train_en_tok[0][:20])
# print(train_fr_tok[0][:20])

# # Khởi tạo mô hình (dùng kích thước từ vocab đã tạo)
# encoder = Encoder(len(src_vocab), EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
# decoder = Decoder(len(tgt_vocab), EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS, DROPOUT)
# model = Seq2Seq(encoder, decoder, device).to(device)

# # Optimizer và loss
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# # Use ignore_index for PAD_TOKEN in CrossEntropyLoss
# criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab.pad_idx)


# # (Phần training vẫn như cũ)
# best_val_loss = float('inf')
# patience = 3
# patience_counter = 0

# for epoch in range(NUM_EPOCHS):
#     train_loss = train_epoch(model, train_loader, optimizer, criterion, CLIP, device)
#     val_loss = evaluate(model, val_loader, criterion, device)

#     print(f'Epoch {epoch+1}/{NUM_EPOCHS}')
#     print(f'Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f}')

#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         patience_counter = 0
#         torch.save(model.state_dict(), 'best_model.pth')
#     else:
#         patience_counter += 1
#         if patience_counter >= patience:
#             print("Early stopping!")
#             break

# bleu_score = calculate_bleu(model, test_data, src_vocab, tgt_vocab, en_tokenizer, device)
# print(f'BLEU Score: {bleu_score:.4f}')



Tokenizing...
Building vocabularies...
Train examples: 29000
Val examples: 1014
Test examples: 1071
Example tokenized (src -> tgt):
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']
['deux', 'jeunes', 'hommes', 'blancs', 'sont', 'dehors', 'près', 'de', 'buissons', '.']
Epoch 1/30
Train Loss: 4.895 | Val Loss: 4.676
Epoch 2/30
Train Loss: 4.033 | Val Loss: 4.382
Epoch 3/30
Train Loss: 3.604 | Val Loss: 4.088
Epoch 4/30
Train Loss: 3.323 | Val Loss: 3.995
Epoch 5/30
Train Loss: 3.071 | Val Loss: 3.863
Epoch 6/30
Train Loss: 2.888 | Val Loss: 3.773
Epoch 7/30
Train Loss: 2.741 | Val Loss: 3.715
Epoch 8/30
Train Loss: 2.598 | Val Loss: 3.671
Epoch 9/30
Train Loss: 2.459 | Val Loss: 3.652
Epoch 10/30
Train Loss: 2.353 | Val Loss: 3.601
Epoch 11/30
Train Loss: 2.279 | Val Loss: 3.559
Epoch 12/30
Train Loss: 2.193 | Val Loss: 3.476
Epoch 13/30
Train Loss: 2.104 | Val Loss: 3.552
Epoch 14/30
Train Loss: 2.012 | Val Loss: 3.587
Epoch 15/30
Train Loss: 1.94

In [ ]:
sentence = " A man is riding a horse."
translation = translate_sentence(model, sentence, src_vocab, tgt_vocab, en_tokenizer, device)
print(f'English: {sentence}')
print(f'French: {translation}')

English:  A man is riding a horse.
French: un homme est sur cheval cheval .
